***
### Read in structure with `create_builder_from_file`

In [ ]:

# ! Way too complex with builder, do it via ase reading in QE input
def add_structure_data(df_in):
    df_out = df_in.copy(deep=True)
    # iterate over tuples of dataframe rows
    structure_datas = []
    for index, row in df_out.iterrows():
        # df_out.at[index, 'structureData'] = 
        try:
            created_builder = create_builder_from_file(
                input_folder=row['abs_path'],
                input_file_name=row['filename'],
                code='qe-6.4.1-pw',
                metadata={},
                pseudo_folder_path=pseudo_dir
                )
            structure_data = created_builder['structure']
        except FileNotFoundError:
            # ! Missing PP (for Ni, PBEsol USPP)
            structure_data = None
        except ParsingError:
            # ! No atomic coordinates in input file, e.g. for DOS
            structure_data = None
        except ValueError:
            print(repr(row['abs_path']))
            print(row['filename'])
            structure_data = None
            raise
        
        structure_datas.append(structure_data)
    df_out['structuredata'] = structure_datas 
    return df_out
    
devel_df = spinel_df_iurii.loc[spinel_df_iurii['abs_path'].str.contains('LixMn2O4/DFT\+U/LiMn2O4/1_vcrelax')]
devel_df = spinel_df_iurii.copy(deep=True)
devel_df.shape
print(devel_df['abs_path'])

devel_df = add_structure_data(devel_df)

devel_df.head()

In [ ]:
# pprint(hubbard_data.hubbard_parameters)
# print(hubbard_data.get_quantum_espresso_hubbard_card())
# print(type(hubbard_data))

# """Create a clone of the given structure but with new kinds, based on the new hubbard sites.

# :param hubbard_structure: ``HubbardStructureData`` instance.
# :param hubbard: the ``hubbard`` output Dict node of a ``HpCalculation``.
# :param magnetization: Dict instance containing the `starting_magnetization` QuantumESPRESSO inputs.
# :return:
# """

# result = structure_relabel_kinds(
#     self.ctx.current_hubbard_structure, workchain.outputs.hubbard, self.ctx.current_magnetic_moments
# )

# hubbard_structure = last_hpcalculation.outputs.hubbard_structure # hubbard structure of HpCalculation, 1. required argument
# hubbard = last_hpcalculation.outputs.hubbard # hubbard dict of HpCalculation, 2. required argument

In [ ]:

hubbard_structure = load_node(3075)
hp_workchain = load_node(4560)

hubbard_structure_outside = hp_workchain.outputs.hubbard_structure
hubbard_dict_outside = hp_workchain.outputs.hubbard

def structure_relabel_kinds_print(hubbard_structure, hubbard_dict, magnetization=None):
    """Create a clone of the given structure but with new kinds, based on the new hubbard sites.

    :param hubbard_structure: ``HubbardStructureData`` instance.
    :param hubbard: the ``hubbard`` output Dict node of a ``HpCalculation``.
    :param magnetization: Dict instance containing the `starting_magnetization` QuantumESPRESSO inputs.
    :return:
    """
    # reference = hubbard_structure.clone()
    # # reference.reorder_atoms()  # we reorder the atoms with Hubbard atoms first

    # relabeled = reference.clone()
    # relabeled.clear_kinds()
    # relabeled.clear_sites()

    # kind_suffix = -1
    # type_to_kind = {}

    # if magnetization:
    #     old_magnetization = magnetization.get_dict()
    #     new_magnetization = old_magnetization.copy()
    #     # Removing old Hubbard spin-polarized atom label.
    #     for site in hubbard_dict["sites"]:
    #         new_magnetization.pop(site["kind"], None)

    # # First do the Hubbard sites, popping the kind name suffix each time a new type is encountered. We do the suffix
    # # generation ourselves, because the indexing done by hp.x contains gaps in the sequence.
    # for index, site in enumerate(hubbard_dict["sites"]):

    #     symbol = re.search(r"^([A-za-z]+)[0-9]*$", site["kind"]).group(1)
    #     try:
    #         # We define a `spin_type`, since hp.x does not distinguish new types according to spin
    #         spin_type = str(int(site["new_type"]) * int(site["spin"]))
    #         kind_name = type_to_kind[spin_type]
    #     except KeyError:
    #         kind_suffix += 1
    #         kind_name = f"{symbol}{kind_suffix}"
    #         type_to_kind[spin_type] = kind_name
    #         if magnetization:
    #             # filling 'starting magnetization' with input value but new label;
    #             # if does not present a starting value, pass.
    #             try:
    #                 new_magnetization[kind_name] = old_magnetization[site["kind"]]
    #             except KeyError:
    #                 pass

    #     site = sites[index]
    #     # relabeled.append_atom(position=site.position, symbols=symbol, name=kind_name)

    sites = hubbard_structure.sites
    # Now add the non-Hubbard sites
    for isite, site in enumerate(sites):

        symbols = hubbard_structure.get_kind(site.kind_name).symbols
        names = hubbard_structure.get_kind(site.kind_name).name
        if isite in relabel_dict.keys():
            names = relabel_dict[isite]

        print(symbols, names)

        # print(relabeled)
        relabeled.append_atom(position=site.position, symbols=symbols, name=names)

    # outputs = {"hubbard_structure": relabeled}
    # if magnetization:
    #     outputs.update({"starting_magnetization": Dict(new_magnetization)})

    # return outputs

def structure_relabel_kinds_mod(hubbard_structure, hubbard_dict, magnetization=None):
    """Create a clone of the given structure but with new kinds, based on the new hubbard sites.

    :param hubbard_structure: ``HubbardStructureData`` instance.
    :param hubbard: the ``hubbard`` output Dict node of a ``HpCalculation``.
    :param magnetization: Dict instance containing the `starting_magnetization` QuantumESPRESSO inputs.
    :return:
    """
    reference = hubbard_structure.clone()
    relabeled = reference.clone()

    relabeled.clear_kinds()
    relabeled.clear_sites()

    kind_suffix = -1
    type_to_kind = {}
    sites = hubbard_structure.sites
    if magnetization is not None:
        old_magnetization = magnetization.get_dict()
        new_magnetization = old_magnetization.copy()
        # Removing old Hubbard spin-polarized atom label.
        for site in hubbard_dict['sites']:
            new_magnetization.pop(site['kind'], None)

    # First do the Hubbard sites, popping the kind name suffix each time a new type is encountered. We do the suffix
    # generation ourselves, because the indexing done by hp.x contains gaps in the sequence.

    # print(type(hubbard_dict['sites']))
    # print(hubbard_dict['sites']) # ! => Returns a list of dicts
    # print(hubbard_structure.sites) # ! => Returns a list of Site objects
    # print(type(hubbard_dict['sites'][0]))
    # print(type(hubbard_structure.sites[0]))
    
    # ! This works now with HubbardStructureData, but ultimately should work with only StructureData
    # ! => Already works with StructureData because it just calls the sites property of the StructureData parent class
    
    final_structuredata = StructureData()
    
    for index, site in enumerate(hubbard_structure.sites):
        # print(index, site)
        raw_site = site.get_raw()
        
        loop_site = deepcopy(site)
        print(type(loop_site))
        loop_site.kind_name = relabel_dict.get(index, site)
        
        print(loop_site)
        print(type(loop_site.kind_name), loop_site.kind_name)
        print(type(loop_site.kind), loop_site.kind)

        # final_structuredata.append_kind(loop_site.kind_name)
        # final_structuredata.append_site(loop_site)
        
        # print(raw_site)
        # print(help(site))
        # ! Return site from raw: Site(raw_site)
        # raise SystemExit
        # symbol = re.search(r'^([A-za-z]+)[0-9]*$', site['kind']).group(1)
        # try:
        #     # We define a `spin_type`, since hp.x does not distinguish new types according to spin
        #     spin_type = str(int(site['new_type']) * int(site['spin']))
        #     kind_name = type_to_kind[spin_type]
        # except KeyError:
        #     kind_suffix += 1
        #     kind_name = f'{symbol}{kind_suffix}'
        #     type_to_kind[spin_type] = kind_name
        #     if magnetization is not None:
        #         # filling 'starting magnetization' with input value but new label;
        #         # if does not present a starting value, pass.
        #         try:
        #             new_magnetization[kind_name] = old_magnetization[site['kind']]
        #         except KeyError:
        #             pass

    # for index, site in enumerate(hubbard_dict['sites']):
    #     print(index, site)
    #     symbol = re.search(r'^([A-za-z]+)[0-9]*$', site['kind']).group(1)
    #     try:
    #         # We define a `spin_type`, since hp.x does not distinguish new types according to spin
    #         spin_type = str(int(site['new_type']) * int(site['spin']))
    #         kind_name = type_to_kind[spin_type]
    #     except KeyError:
    #         kind_suffix += 1
    #         kind_name = f'{symbol}{kind_suffix}'
    #         type_to_kind[spin_type] = kind_name
    #         if magnetization is not None:
    #             # filling 'starting magnetization' with input value but new label;
    #             # if does not present a starting value, pass.
    #             try:
    #                 new_magnetization[kind_name] = old_magnetization[site['kind']]
    #             except KeyError:
    #                 pass

        # loop_site = sites[index]
        # relabeled.append_atom(position=loop_site.position, symbols=symbol, name=kind_name)

    # # Now add the non-Hubbard sites
    # for site in sites[len(relabeled.sites):]:
    #     symbols = hubbard_structure.get_kind(site.kind_name).symbols
    #     names = hubbard_structure.get_kind(site.kind_name).name
    #     relabeled.append_atom(position=site.position, symbols=symbols, name=names)

    # outputs = {'hubbard_structure': relabeled}
    # if magnetization is not None:
    #     outputs.update({'starting_magnetization': Dict(new_magnetization)})

    # return outputs

# print(hubbard_structure.get_quantum_espresso_hubbard_card())

structure_relabel_kinds_print(
    hubbard_structure=hubbard_structure_outside,
    hubbard_dict=hubbard_dict_outside,
    magnetization=None
)

structure_relabel_kinds_mod(
    hubbard_structure=hubbard_structure_outside,
    hubbard_dict=hubbard_dict_outside,
    magnetization=None
)

In [ ]:
# ! Trying to append special Mn1 and Mn2 labels in ase.Atoms

# ! Trying to append special Mn1 and Mn2 labels in pmg.Structure

# ! Trying to append special Mn1 and Mn2 labels in the structuredata
# for isite, site in enumerate(devel_structuredata.sites):
#     temp_site = deepcopy(site)
#     if isite in (0, 2):
#         temp_site.kind_name = 'Mn1'
#     elif isite in (1, 3):
#         temp_site.kind_name = 'Mn2'
#     new_devel_structuredata.append_site(temp_site)


# print(devel_structuredata.kinds)
# print(devel_structuredata.sites)
# print(devel_structuredata.get_site_kindnames())


# mn_olivine_pmg.replace(0, 'Mn1')
# mn_olivine_pmg.replace(1, 'Mn2')
# mn_olivine_pmg.replace(2, 'Mn1')
# mn_olivine_pmg.replace(3, 'Mn2')
# print(mn_olivine_pmg.formula)

# Add magnetization to pmg Structure
# mn_olivine_pmg.add_site_property("magmom", [0.5, 0.5, -0.5, -0.5]+[0]*(mn_olivine_pmg.num_sites-4))


# print(mn_olivine_pmg)

# hubbard_data_inv = HubbardStructureData(structure=mn_olivine_aiida)
# hubbard_data_inv.initialize_onsites_hubbard('Mn', '3d', 4.5618)
# hubbard_data_inv.hubbard_parameters
# hubbard_data_inv.initialize_intersites_hubbard('Mn', '3d', 'O', '2p', 0.0001, number_of_neighbours=7) 